In [ ]:
import os
import cv2
import json
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

JSON_PATH = "football_train/Match_1951_1_0_subclip/Match_1951_1_0_subclip.json"
IMG_PATH = "frames/Match_1951_1_0_subclip"

with open(JSON_PATH, "r") as file:
    data = json.load(file)

In [ ]:
for image_data in data['images']:
    file_name = image_data['file_name']
    image_id = image_data['id']

    img = cv2.imread(filename=os.path.join(IMG_PATH, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    
    annot_lst = [annot for annot in data['annotations'] if annot['image_id'] == image_id and annot['category_id'] == 4]

    for annot in annot_lst:
        bbox = annot['bbox']
        player_number = annot['attributes']['jersey_number']

        x, y, w, h = map(int, bbox) 

        color = (0, 255, 0)  # Green color
        thickness = 2
        cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)

        (text_width, text_height), _ = cv2.getTextSize(
            str(player_number), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            1
            )

        cv2.putText(
            img, 
            str(player_number), 
            (x, y - 5), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            2, 
            (0, 0, 0), 
            5, cv2.LINE_AA)
        # cropped_image = img[y:y+h, x:x+w]
        
    plt.figure(figsize=(10,5))
    plt.axis("off")
    plt.imshow(img)

    break

In [ ]:
for image_data in data['images']:
    file_name = image_data['file_name']
    image_id = image_data['id']

    img = cv2.imread(filename=os.path.join(IMG_PATH, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    
    annot_lst = [annot for annot in data['annotations'] if annot['image_id'] == image_id and annot['category_id'] == 4]

    for annot in annot_lst:
        # Crop image
        bbox = annot['bbox']
        x, y, w, h = map(int, bbox) 
        cropped_image = img[y:y+h, x:x+w]

        # Get player number
        player_number = annot['attributes']['jersey_number']
        
        plt.figure(figsize=(3,3))
        plt.axis("off")
        plt.imshow(cropped_image)
        plt.title(f"number: {player_number}")

    break

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class FootBall(Dataset):
    def __init__(self, file_name:str):
        super().__init__()

        self.json_path = f"football_train/{file_name}/{file_name}.json"
        self.image_path = f"frames/{file_name}"

        with open(self.json_path, "r") as file:
            self.data = json.load(file)
        
        self.all_images = list()
        self.numbers = list()

        for image_data in tqdm(data['images']):
            crop_imgs, player_nums = self._get_crop_imgs_with_player_numbers(image_data=image_data)

            self.all_images.append(crop_imgs)
            self.numbers.append(player_nums)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.all_images[index], self.numbers[index]

    
    def _get_crop_imgs_with_player_numbers(self, image_data:dict) -> tuple:
        crop_imgs = list()
        player_nums = list()
        
        file_name = image_data['file_name']
        image_id = image_data['id']

        img = cv2.imread(filename=os.path.join(self.image_path, file_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)

        annot_lst = [annot for annot in self.data['annotations'] if annot['image_id'] == image_id and annot['category_id'] == 4]
        for annot in annot_lst:
            # Crop image
            bbox = annot['bbox']
            x, y, w, h = map(int, bbox) 
            cropped_image = img[y:y+h, x:x+w]

            # Get player number
            player_number = annot['attributes']['jersey_number']

            crop_imgs.append(cropped_image)
            player_nums.append(player_number)
        
        return crop_imgs, player_nums
        

In [ ]:
data = FootBall(file_name="Match_1951_1_0_subclip")

In [ ]:
class FootBall(Dataset):
    def __init__(self, file_name:str):
        super().__init__()

        self.json_path = f"football_train/{file_name}/{file_name}.json"
        self.image_path = f"frames/{file_name}"

        with open(self.json_path, "r") as file:
            self.data = json.load(file)
        
        self.images = self.data['images']
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_data = self.images[index]
        crop_imgs, player_nums = self._get_crop_imgs_with_player_numbers(image_data=image_data)
        return crop_imgs, player_nums

    def _get_crop_imgs_with_player_numbers(self, image_data:dict) -> tuple:
        crop_imgs = list()
        player_nums = list()
        
        file_name = image_data['file_name']
        image_id = image_data['id']

        img = cv2.imread(filename=os.path.join(self.image_path, file_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        annot_lst = [annot for annot in self.data['annotations'] if annot['image_id'] == image_id and annot['category_id'] == 4]
        for annot in annot_lst:
            # Crop image
            bbox = annot['bbox']
            x, y, w, h = map(int, bbox) 
            cropped_image = img[y:y+h, x:x+w]

            # Get player number
            player_number = annot['attributes']['jersey_number']

            crop_imgs.append(cropped_image)
            player_nums.append(player_number)
        
        return crop_imgs, player_nums

In [ ]:
data = FootBall(file_name="Match_1951_1_0_subclip")

In [ ]:
images, numbers = data[0]

In [ ]:
for image in images:
    print(image.shape)

In [ ]:
len(images)

In [ ]:
numbers

In [ ]:
len(numbers)